In [196]:
import pandas as pd
import numpy as np

In [197]:
db_ranking = pd.read_csv('dbRanking.csv', delimiter='|')
db_ranking.sample(5)

,taxpayer_id,company_economic_activity_id,ciiu_spanish_desc,spanish_sector_company_name,employees_number,branches_number,activity_start_date,seniority_company_years_number,company_size_desc,foreign_company_mark_type,...,pbi_sector_2015_per,pbi_sector_2014_per,pbi_sector_2013_per,pbi_sector_2012_per,pbi_sector_2011_per,pbi_sector_2010_per,pbi_sector_2009_per,pbi_sector_2008_per,pbi_sector_2007_per,cutoff_date
5663,90848538481,5122,"Venta al por mayor de alimentos, bebidas y tabaco",COMERCIALIZADORAS,18.0,1.0,1997-04-15,21.0,Mediana empresa / Medium-size Company,NaN,...,1.8,3.9,4.4,5.8,6.70,8.80,9.69,-0.9,12.80,2020-02-28
8440,31996258764,5030,"Ventas de partes, piezas y accesorios de vehÃ­...",VEHICULOS,18.0,2.0,2002-09-28,16.0,Mediana empresa / Medium-size Company,NaN,...,1.8,5.5,4.9,5.9,6.71,8.82,9.69,-0.4,12.99,2020-02-28
12907,21730729699,1320,ExtracciÃ³n de minerales matalÃ­feros no ferro...,MINERIA,57.0,1.0,2005-04-22,13.0,PequeÃ±a empresa / Small-size Company,NaN,...,1.8,3.9,4.4,5.8,6.70,8.80,9.69,-0.9,12.80,2020-02-28
4718,3672820819,5030,"Ventas de partes, piezas y accesorios de vehÃ­...",VEHICULOS,10.0,1.0,1990-08-01,28.0,Mediana empresa / Medium-size Company,NaN,...,1.8,5.5,4.9,5.9,6.71,8.82,9.69,-0.4,12.99,2020-02-28
11805,21226409163,5040,"Venta, mantenimiento y reparacion de motocicle...",VEHICULOS,3.0,1.0,2004-01-12,14.0,PequeÃ±a empresa / Small-size Company,NaN,...,1.8,5.5,4.9,5.9,6.71,8.82,9.69,-0.4,12.99,2020-02-28


In [198]:
ranking_dict = ['ranking_'+str(i)+'_number' for i in range(2017,2009,-1)]
min_usd_billing_dict = ['min_usd_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
max_usd_billing_dict = ['max_usd_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
min_pen_billing_dict = ['min_pen_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)] 
max_pen_billing_dict = ['max_pen_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
pbi_sector_dict = ['pbi_sector_'+str(i)+'_per' for i in range(2016,2006,-1)]

def clean_db_ranking(df):
    df = df.drop_duplicates(ignore_index=True)
    df = df.dropna(axis=1, how='all')
    df = df[df['market_share_per'] < 99] # Removing, these are like 10 registers
    df = df.drop(['ebitda_2014_amount', 'ebitda_2013_amount'], axis=1) # Removing these columns because are too few
    
    for col in ranking_dict: #computing and droping ranking
        df['ranking_2017_number'].fillna(df[col], inplace=True)
    df['ranking_last_number'] = df['ranking_2017_number']
    df = df.drop(ranking_dict, axis=1)
    
    for col in min_usd_billing_dict: #computing and dropping min usd amounts
        df['min_usd_billing_2017_amount'].fillna(df[col], inplace=True)
    df['min_usd_billing_last_amount'] = df['min_usd_billing_2016_amount']
    df = df.drop(min_usd_billing_dict, axis=1)

    for col in max_usd_billing_dict: #computing and dropping max usd amounts
        df['max_usd_billing_2017_amount'].fillna(df[col], inplace=True)
    df['max_usd_billing_last_amount'] = df['max_usd_billing_2017_amount']
    df = df.drop(max_usd_billing_dict, axis=1)

    for col in min_pen_billing_dict: #computing and dropping max usd amounts
        df['min_pen_billing_2017_amount'].fillna(df[col], inplace=True)
    df['min_pen_billing_last_amount'] = df['min_pen_billing_2017_amount']
    df = df.drop(min_pen_billing_dict, axis=1)

    for col in max_pen_billing_dict: #computing and dropping max usd amounts
        df['max_pen_billing_2017_amount'].fillna(df[col], inplace=True)
    df['max_pen_billing_last_amount'] = df['max_pen_billing_2017_amount']
    df = df.drop(max_pen_billing_dict, axis=1)

    df['exporter_mark_type_flag'] = df['exporter_mark_type']
    df['importer_mark_type_flag'] = df['importer_mark_type']

    df = df.drop(df.columns[[i for i in range(15, 185)]], axis=1)

    for col in pbi_sector_dict: #computing and dropping max usd amounts
        df['pbi_sector_2016_per'].fillna(df[col], inplace=True)
    df['pbi_sector_last_per'] = df['pbi_sector_2016_per']
    df['pbi_sector_last_per'] = df['pbi_sector_last_per'].replace(0, np.nan)
    df = df.drop(pbi_sector_dict, axis=1)

    df = df.drop(['cutoff_date'], axis=1)

    return df

In [199]:
db_ranking = clean_db_ranking(db_ranking)
db_ranking.sample(5)

,taxpayer_id,company_economic_activity_id,ciiu_spanish_desc,spanish_sector_company_name,employees_number,branches_number,activity_start_date,seniority_company_years_number,company_size_desc,foreign_company_mark_type,...,private_public_mark_desc,market_share_per,ranking_last_number,min_usd_billing_last_amount,max_usd_billing_last_amount,min_pen_billing_last_amount,max_pen_billing_last_amount,exporter_mark_type_flag,importer_mark_type_flag,pbi_sector_last_per
3796,48058611240,3720,Reciclamiento de desperdicios y desechos no me...,RECICLAJE,13.0,1.0,1993-02-01,25.0,Mediana empresa / Medium-size Company,NaN,...,PRIVADA,6.588313,8517.0,2903704.0,3218883.814,9900000.0,10499999.0,NaN,IMPORTADORA,3.9
3180,87721219230,9211,ProducciÃ³n y distribuciÃ³n de filmes y videoc...,ESPARCIMIENTO,8.0,1.0,1993-12-22,25.0,Mediana empresa / Medium-size Company,NaN,...,PRIVADA,6.672004,7635.0,3659259.0,3770692.520,12000000.0,12299999.0,EXPORTADORA,IMPORTADORA,3.4
2350,87982748466,6720,Actividades auxiliares de la financiaciÃ³n de ...,INTERMEDIACION FINANCIERA,213.0,2.0,1989-02-01,29.0,Mediana empresa / Medium-size Company,NaN,...,PRIVADA,2.059432,3612.0,7170370.0,7510729.307,19000000.0,24499999.0,NaN,IMPORTADORA,6.0
802,90859118891,4520,ConstrucciÃ³n de edificios completos o de part...,CONSTRUCTORAS,5.0,1.0,2004-04-17,14.0,PequeÃ±a empresa / Small-size Company,NaN,...,PRIVADA,0.025214,7712.0,3229630.0,3659258.000,10900000.0,12349999.0,NaN,NaN,NaN
3866,45493849318,2811,FabricaciÃ³n de productos metÃ¡licos para uso ...,METALMECANICA,3.0,1.0,2006-07-01,12.0,PequeÃ±a empresa / Small-size Company,NaN,...,PRIVADA,0.274468,NaN,NaN,NaN,NaN,NaN,EXPORTADORA,IMPORTADORA,NaN


In [172]:
db_ranking['pbi_sector_2016_per'].value_counts()

0.0     2793
2.4     1012
3.4      574
1.8      391
6.8      186
2.0      173
2.5      155
1.0      126
3.0      117
4.0       88
6.0       85
1.9       40
4.6       16
10.0      11
6.9        5
3.9        3
4.8        1
Name: pbi_sector_2016_per, dtype: int64